# Question 1

In [1]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

user_question = "I just discovered the course. Can I still join it?"

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Generate the embeddings
embedding_vector = embedding_model.encode(user_question)
v = embedding_model.encode(user_question)

# Get the first value of the resulting vector
v = embedding_vector[0]

print(v)

0.078222655


# Question 2

In [3]:
import requests 

docs_url = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-vector-search/eval/documents-with-ids.json'
docs_response = requests.get(docs_url)
documents = docs_response.json()


ml_zoomcamp_docs = []

for doc in documents:
    if doc['course'] == 'machine-learning-zoomcamp':
        ml_zoomcamp_docs.append(doc)


In [4]:
for doc in (ml_zoomcamp_docs):
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'

    doc['question_vector'] =embedding_model.encode(question)
    doc['text_vector'] =embedding_model.encode(text)
    doc['question_text_vector'] = embedding_model.encode(qa_text)

In [6]:
import numpy as np
embeddings = []

# Create embeddings for each document
for doc in ml_zoomcamp_docs:
    qa_text = f"{doc['question']} {doc['text']}"
    embedding = embedding_model.encode(qa_text)
    embeddings.append(embedding)

# Convert to a NumPy array and print shape
X = np.array(embeddings)
print(X.shape)

(375, 768)


# Question 3

In [7]:
scores = X.dot(v)

# Find the highest score
highest_score = np.max(scores)

print(highest_score)

0.012700376


# Question 4

In [8]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=ml_zoomcamp_docs, embeddings=X)
# search_engine.search(embedding_vector, num_results=5)

In [9]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [10]:
relevance_total = []

for q in (ground_truth):
    doc_id = q['document']
    question_embeddings = embedding_model.encode(q['question'])
    results = search_engine.search(question_embeddings, num_results=5)

    relevance = [d['id'] in q['document'] for d in results]
    relevance_total.append(relevance)

In [11]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

hit_rate(relevance_total)

0.9398907103825137

# Question 5

In [12]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '711ba8d8e466', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'hFOHHyYZRpO2MGPHdBv3Hg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [29]:
es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [45]:
for doc in (ml_zoomcamp_docs):
    es_client.index(index=index_name, document=doc)

In [46]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )

    result_docs = []

    for hit in es_results['hits']['hits']:
        result_docs.append((hit['_source']['id'],hit['_score']))

    return result_docs

In [54]:
# Call the elastic_search_knn function
results = elastic_search_knn('question_text_vector', question_embedding, 'machine-learning-zoomcamp')

# Check if there are results
if results:
    highest_id, highest_score = results[0]  # Get the first document ID and score
    print(f'ID of the document with the highest score: {highest_id}')
else:
    print('No results found.')


ID of the document with the highest score: ee58a693


# Question 6

In [59]:
def question_vector_knn(q):
    question = q['question']
    course = q['course']
    v_q = embedding_model.encode(question)
    return elastic_search_knn('question_text_vector', v_q, course)

relevance_total_elastic = []

for q in ground_truth:
    doc_id = q['document']
    results = question_vector_knn(q)

    # Check if the retrieved document IDs match the expected document ID
    relevance = [d[0] == doc_id for d in results]  # d[0] accesses the document ID
    relevance_total_elastic.append(relevance)


In [58]:
hit_rate(relevance_total_elastic)

0.9398907103825137